In [4]:
pip install py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 200 kB 5.6 MB/s 


In [9]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark


In [12]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.0'
os.environ['SPARK_VERSION']=spark_version

In [13]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [14]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

In [15]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-14 21:19:28--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-08-14 21:19:29 (11.4 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [25]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [26]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [32]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20.
new_total_votes_df = df.filter(df.total_votes >= 20)
new_total_votes_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33460969|R28RB82UG4RDD5|B00FB4UPA0|      10711472|Nifty Home 24 K-C...|         Kitchen|          5|           20|         20|   N|                Y| Saves counter space|Fits under my ful...| 2015-08-31|
|         US|   47272965|R3FB6BERWPEIJP|B0029OXHVY|     236056882|Wusthof Classic I...|         Kitchen|          4|    

In [33]:
# Filter the new DataFrame  and create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
votes_greater_df = new_total_votes_df.filter((new_total_votes_df.helpful_votes / new_total_votes_df.total_votes) >= 0.5)
votes_greater_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33460969|R28RB82UG4RDD5|B00FB4UPA0|      10711472|Nifty Home 24 K-C...|         Kitchen|          5|           20|         20|   N|                Y| Saves counter space|Fits under my ful...| 2015-08-31|
|         US|   47272965|R3FB6BERWPEIJP|B0029OXHVY|     236056882|Wusthof Classic I...|         Kitchen|          4|    

In [35]:
# Filter new df and create a new DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_vine_df = votes_greater_df.filter(votes_greater_df.vine == 'Y')
paid_vine_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33146712|R1Z71RW4J9IK93|B00YCP6UEO|     238165328|De'Longhi Automat...|         Kitchen|          5|           20|         22|   Y|                N|If you make cappu...|It's really hard ...| 2015-08-29|
|         US|   16449486|R3FVB5QI11KI9Q|B00YCP71VK|      62048996|De'Longhi ECP3220...|         Kitchen|          4|    

In [36]:
# Retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_vine_df =votes_greater_df.filter(votes_greater_df.vine == 'N')
unpaid_vine_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33460969|R28RB82UG4RDD5|B00FB4UPA0|      10711472|Nifty Home 24 K-C...|         Kitchen|          5|           20|         20|   N|                Y| Saves counter space|Fits under my ful...| 2015-08-31|
|         US|   47272965|R3FB6BERWPEIJP|B0029OXHVY|     236056882|Wusthof Classic I...|         Kitchen|          4|    

In [37]:
# Determine the total number of reviews for the PAID review.
total_paid_reviews = paid_vine_df.count()
total_paid_reviews

2287

In [38]:
# Determine the number of 5-star reviews for the PAID review.
total_paid_five_start_reviews = paid_vine_df.filter(paid_vine_df.star_rating == 5).count()
total_paid_five_start_reviews

963

In [39]:

# Determine the percentage of 5-star reviews for the PAID review.
percentage_paid_five_star_reviews = (total_paid_five_start_reviews/total_paid_reviews) * 100
percentage_paid_five_star_reviews

42.10756449497158

In [40]:
# Determine the total number of reviews for the UNPAID review.
total_unpaid_reviews = unpaid_vine_df.count()
total_unpaid_reviews

147512

In [41]:
# Determine the number of 5-star reviews for the UNPAID review.
total_unpaid_five_start_reviews = unpaid_vine_df.filter(unpaid_vine_df.star_rating == 5).count()
total_unpaid_five_start_reviews

68901

In [42]:
# Determine the percentage of 5-star reviews for the UNPAID review.
percentage_unpaid_five_star_reviews = (total_unpaid_five_start_reviews/total_unpaid_reviews) * 100
percentage_unpaid_five_star_reviews

46.70874233960627

In [43]:
import pandas as pd

In [44]:
paid_vine_df.agg({"total_votes": "mean"}).show()

+-----------------+
| avg(total_votes)|
+-----------------+
|88.60996939221688|
+-----------------+



In [45]:
paid_vine_df.agg({"total_votes": "variance"}).show()

+---------------------+
|variance(total_votes)|
+---------------------+
|    29514.33862322741|
+---------------------+



In [46]:
paid_vine_df.agg({"total_votes": "stddev"}).show()

+-------------------+
|stddev(total_votes)|
+-------------------+
|  171.7973766482696|
+-------------------+

